# Introduction to pyNS 
pyNS is a Python library to programmatically access the Neuroscout API. pyNS let's you query the API and create analyses, without having to mess around with buildling any JSON requests yourself.

In this tutorial, I'll demonstrate how to query the API to create your own analysis.

In [74]:
from pyns import Neuroscout
api = Neuroscout('user@university.edu', 'yourpassword')

The `Neuroscout` object will be your main entry point to the API. You can instantiate this object without any credentials to access public API routes, or preferably with your Neuroscout credentials to be able to create and save your analyses. The `Neuroscout` object has links to each main API route, and each of these links implements the standard HTTP verbs that are suppoted by each route, such as `datasets`, `runs`, `predictors`, etc...

### Querying datasets and runs

In [2]:
api.datasets.get()

[{'description': {'Authors': ['Alexander, L. et al.'],
   'BIDSVersion': '1.0.2',
   'Name': 'Healthy_Brain_Network'},
  'id': 8,
  'mimetypes': ['audio/x-wav', 'image/png', 'text/plain', 'video/x-matroska'],
  'name': 'HealthyBrainNetwork',
  'runs': [216,
   200,
   202,
   204,
   205,
   206,
   208,
   209,
   210,
   211,
   212,
   213,
   214,
   215,
   217,
   218,
   219,
   220,
   221,
   222,
   223,
   224,
   225,
   226,
   227,
   228,
   229,
   230,
   231,
   399,
   400,
   401,
   402,
   403,
   404,
   405,
   406,
   407,
   408,
   409,
   410,
   411,
   412,
   413,
   414,
   415,
   416,
   417,
   418,
   419,
   420,
   421,
   422,
   423,
   424,
   425,
   426,
   427,
   428,
   429],
  'summary': 'Movie watching paradigms from the Healthy Brain Network brain development study',
  'tasks': [{'id': 7,
    'name': 'movieDM',
    'num_runs': 60,
    'summary': 'AV Presentation: Despicable Me'}],
  'url': 'http://fcon_1000.projects.nitrc.org/indi/cmi_he

This request returns a list of two datasets, with information about the dataset, as well as the run IDs associated with this dataset. Let's focus on the dataset, `life`

If we want more information on the specific runs within this dataset, we can query the `runs` route, using the dataset_id associated with `life`

In [16]:
dataset = api.datasets.get()[1]
api.runs.get(dataset_id=dataset['id'])

[{'acquisition': '346vol',
  'dataset_id': 9,
  'duration': 865.0,
  'id': 235,
  'number': 2,
  'session': None,
  'subject': 'rid000001',
  'task': 8},
 {'acquisition': '374vol',
  'dataset_id': 9,
  'duration': 935.0,
  'id': 236,
  'number': 1,
  'session': None,
  'subject': 'rid000001',
  'task': 8},
 {'acquisition': '377vol',
  'dataset_id': 9,
  'duration': 942.5,
  'id': 237,
  'number': 3,
  'session': None,
  'subject': 'rid000001',
  'task': 8},
 {'acquisition': '412vol',
  'dataset_id': 9,
  'duration': 1030.0,
  'id': 238,
  'number': 4,
  'session': None,
  'subject': 'rid000001',
  'task': 8},
 {'acquisition': '346vol',
  'dataset_id': 9,
  'duration': 865.0,
  'id': 239,
  'number': 2,
  'session': None,
  'subject': 'rid000005',
  'task': 8},
 {'acquisition': '374vol',
  'dataset_id': 9,
  'duration': 935.0,
  'id': 240,
  'number': 1,
  'session': None,
  'subject': 'rid000005',
  'task': 8},
 {'acquisition': '377vol',
  'dataset_id': 9,
  'duration': 942.5,
  'id': 

Using this information, we can decide which runs to focus on for our analysis.

### Querying predictors

Now let's take a look at the predictors associated with this dataset

In [18]:
api.predictors.get(run_id=dataset['runs'])

[{'description': 'Clarifai image recognition label: horizontal',
  'extracted_feature': {'created_at': '2018-05-09 22:42:17.995043',
   'description': 'Clarifai image recognition label: horizontal',
   'extractor_name': 'ClarifaiAPIImageExtractor',
   'id': 13660,
   'modality': None},
  'id': 12800,
  'max': 0.95819473,
  'mean': 0.4212714761726175,
  'min': 0.0149346925,
  'name': 'horizontal',
  'num_na': 0,
  'source': 'extracted',
  'stddev': 0.19377647775118148},
 {'description': 'Clarifai image recognition label: hand',
  'extracted_feature': {'created_at': '2018-05-09 22:42:17.976167',
   'description': 'Clarifai image recognition label: hand',
   'extractor_name': 'ClarifaiAPIImageExtractor',
   'id': 13658,
   'modality': None},
  'id': 12798,
  'max': 0.6226746,
  'mean': 0.039579114216090575,
  'min': 1.0978953e-05,
  'name': 'hand',
  'num_na': 0,
  'source': 'extracted',
  'stddev': 0.07129718248813187},
 {'description': 'Clarifai image recognition label: desktop',
  'ext

A bunch of useful information to help me choose some features! Let's keep it simple and go with 'rmse' (sound volume) and 'FramewiseDisplacement':

## Building an Analysis

Now, let's build an analysis. For this, we can use the `Analysis` class, which makes it easy to build an Analysis locally, by mirroring the Analysis representation on the server. 

To build an `Analysis` object, we can use the `create_analysis` which pre-populates our `Analysis` object with the relevant information, including a pre-build BIDS model, and registers it to the API. 

In [30]:
analysis = api.analyses.create_analysis(
    dataset_name='Life', name='My new analysis!',
    predictor_names=['rmse', 'FramewiseDisplacement'],
    hrf_variables=['rmse'], 
    subject=['rid000001', 'rid000005']
)

This newly created analysis has been assigned a unique ID by the Neuroscout API

In [31]:
analysis.hash_id

'9B8pA'

In [32]:
# Some properties are read-only and came from the server
analysis.created_at

'2019-01-24T21:5'

The analysis creation function has found the runs relevant to the subjects we're interested in, and created a basic BIDS-Model for our analysis:

In [33]:
analysis.model

{'Input': {'Subject': ['rid000001', 'rid000005'], 'Task': 'life'},
 'Name': 'My new analysis!',
 'Steps': [{'AutoContrasts': True,
   'Contrasts': [],
   'Level': 'Run',
   'Model': {'X': ['rmse', 'FramewiseDisplacement']},
   'Transformations': [{'Input': ['rmse'], 'Name': 'Convolve'}]},
  {'AutoContrasts': True, 'Level': 'Subject'},
  {'AutoContrasts': True, 'Level': 'Dataset'}]}

In [34]:
analysis.runs

[235, 236, 237, 238, 239, 240, 241, 242]

In [35]:
# Neuroscout API Predictor IDs
analysis.predictors

[12735, 12904]

We can edit this Analysis object to fill in any other Analysis details, and push them to the Neuroscout API:

In [36]:
analysis.description = "This is my analysis, and it's probably the best"
analysis.push()

### Reports

Now that we have created and design an analysis we can generate some reports based on our design

Let's generate a report using only a single run 

In [37]:
analysis.generate_report(run_id=analysis.runs[0])

{'generated_at': '2019-01-24T21:5',
 'result': None,
 'status': 'PENDING',
 'traceback': None}

This report should take a few seconds to a few minutes to compile, and we can check its status:

In [43]:
report = analysis.get_report(run_id=analysis.runs[0])

In [44]:
report

{'generated_at': '2019-01-24T21:5',
 'result': {'contrast_plot': ['alpha.neuroscout.org/reports/9B8pA/sub-rid000001_task-life_run-2_contrast_matrix.png'],
  'design_matrix': ['alpha.neuroscout.org/reports/9B8pA/sub-rid000001_task-life_run-2_design_matrix.tsv'],
  'design_matrix_corrplot': ['alpha.neuroscout.org/reports/9B8pA/sub-rid000001_task-life_run-2_design_matrix_corrplot.png'],
  'design_matrix_plot': ['alpha.neuroscout.org/reports/9B8pA/sub-rid000001_task-life_run-2_design_matrix_plot.png']},
 'status': 'OK',
 'traceback': None}

In [48]:
report

{'generated_at': '2019-01-24T21:5',
 'result': {'contrast_plot': ['alpha.neuroscout.org/reports/9B8pA/sub-rid000001_task-life_run-2_contrast_matrix.png'],
  'design_matrix': ['alpha.neuroscout.org/reports/9B8pA/sub-rid000001_task-life_run-2_design_matrix.tsv'],
  'design_matrix_corrplot': ['alpha.neuroscout.org/reports/9B8pA/sub-rid000001_task-life_run-2_design_matrix_corrplot.png'],
  'design_matrix_plot': ['alpha.neuroscout.org/reports/9B8pA/sub-rid000001_task-life_run-2_design_matrix_plot.png']},
 'status': 'OK',
 'traceback': None}

Great, our report was sucesfully generated with no errors. Now lets take a look at the resulting design matrix:

In [56]:
from IPython.display import Image
Image(url="https://" + report['result']['design_matrix_plot'][0])

### Compiling the analysis
Finally, now that we are happy with our analysis, we can ask Neuroscout to verify the analysis, and generate an analysis bundle for us

In [57]:
analysis.compile()

{'compile_traceback': '', 'status': 'PENDING'}

In [58]:
analysis.get_status()

{'compile_traceback': '', 'status': 'PASSED'}

Great! Our analysis passed with no errors. We can now run our analysis using the `neuroscout-cli`. 

For more information on the `neuroscout-cli`, see here: https://github.com/neuroscout/neuroscout-cli

### Cloning our analysis

Now that we've gone off and run our analysis, we realized we want to make some changes. In this case, I'm just going to change the analysis name.

With Neuroscout this is easy, because I simply clone my previous analysis, and take off from I left off

In [59]:
new_analysis = analysis.clone()
new_analysis.hash_id

'MDOOA'

In [60]:
new_analysis.name = 'My new analysis name!'

However, what if we wanted to take this same model, and apply it to a different model. For example, `dataset_id` 5, which correspond to SherlockMerlin?

To do so, we have to use the `fill` function to get the correct `predictors` and `runs`, as these IDS "correspond to the wrong dataset

In [69]:
new_analysis.predictors = []
new_analysis.runs = []
new_analysis.dataset_id = 5

In [70]:
new_analysis.fill()

{'TR': 2.5,
 'compile_traceback': '',
 'created_at': '2019-01-24T21:5',
 'dataset_id': 9,
 'description': "This is my analysis, and it's probably the best",
 'hash_id': 'MDOOA',
 'locked': False,
 'model': {'Input': {'Subject': ['rid000001', 'rid000005'], 'Task': 'life'},
  'Name': 'My new analysis!',
  'Steps': [{'AutoContrasts': True,
    'Contrasts': [],
    'Level': 'Run',
    'Model': {'X': ['rmse', 'FramewiseDisplacement']},
    'Transformations': [{'Input': ['rmse'], 'Name': 'Convolve'}]},
   {'AutoContrasts': True, 'Level': 'Subject'},
   {'AutoContrasts': True, 'Level': 'Dataset'}]},
 'modified_at': '2019-01-24T21:5',
 'name': 'My new analysis!',
 'parent_id': '9B8pA',
 'predictions': '',
 'predictors': [12735, 12904],
 'private': True,
 'runs': [235, 236, 237, 238, 239, 240, 241, 242],
 'status': 'DRAFT',
 'submitted_at': '2019-01-24T22:0',
 'task_name': 'life'}

This function automatically filled in all available runs for dataset_id = 5, and found the corresponding predictor ids based on the names used in the model. We can now compile this cloned analysis.

In [71]:
new_analysis.compile()

{'compile_traceback': '', 'status': 'PENDING'}

In [72]:
new_analysis.get_status()

{'compile_traceback': '', 'status': 'PASSED'}

Great, since this analysis also passed, I can easily run this forked version using the `neuroscout-cli`. If you have any questions on how to use the Neuroscout API, please see the API's Swagger Documentation, which covers all possible routes and parameters: https://alpha.neuroscout.org/swagger-ui